In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
sales_train = pd.read_csv("./data/sales_train.csv")
items = pd.read_csv("./data/items.csv")
shops = pd.read_csv("./data/shops.csv")
item_category = pd.read_csv("./data/item_categories.csv")
sales_test = pd.read_csv("./data/test.csv")

In [3]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


**Data Clean-Up**

Removing the outliers based on item_price and item_cnt_day

In [4]:
sales_train.item_price.sort_values()

484683         -1.0000
608763          0.0700
579766          0.0700
611400          0.0875
673155          0.0900
              ...     
2910156     42990.0000
2327159     49782.0000
1488135     50999.0000
885138      59200.0000
1163158    307980.0000
Name: item_price, Length: 2935849, dtype: float64

In [5]:
sales_train.item_cnt_day.sort_values()

1057907     -22.0
1835454     -16.0
2048519      -9.0
59156        -6.0
940092       -6.0
            ...  
2608040     624.0
2851091     637.0
2864235     669.0
2326930    1000.0
2909818    2169.0
Name: item_cnt_day, Length: 2935849, dtype: float64

from the above values and from our explanatory analyiss, we can clearly see than the outlier for item_price is '50999.0000', '59200.0000',and '307980.0000' and for 'item_cnt_day' is '1000' and '2169.0'.

So we're considering 'item_cnt_day' less than '1000' and 'item_price' less than '50000' and also only positive 'item_price'

And, also the negative values in the 'item_cnt_day' can be items returned to the store, so we're keeping it.

In [6]:
train = sales_train.copy()
test = sales_test.copy()

In [7]:
train = train[(train['item_price'] < 50000 ) & (train['item_price'] > 0) & (train['item_cnt_day'] < 1000) ]

In [8]:
print(f'number of records removed : {(sales_train.shape[0]) - (train.shape[0])} ')

number of records removed : 6 


In [9]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [10]:
item_sale_cnt = train.groupby(['item_id'])['date'].count() # count of sales for each item (item_id)

In [11]:
latest_sale_month = train.groupby(["item_id"])["date_block_num"].max() # latest sale month for each item (item_id)

In [12]:
item_cnt_month_train = pd.concat([item_sale_cnt, latest_sale_month], axis=1)
item_cnt_month_train.columns = ['cnt_per_item', 'latest_sale_date']
item_cnt_month_train.head()

,cnt_per_item,latest_sale_date
item_id,,
0,1,20
1,6,21
2,2,22
3,2,19
4,1,20


Removing the data with sales count for an item is less than 10 and also item that hasn't sold in the last year.

The maximum date_block_num is 33, we're removing the items taht hasn't sold in the last year, so 33 - 12 = 21, so items with max date_block_num less than 22, we'll remove it.

In [13]:
item_to_drop = item_cnt_month_train[(item_cnt_month_train.cnt_per_item < 10) & (item_cnt_month_train.latest_sale_date <= 21)] 
item_to_drop.shape

(4154, 2)

In [14]:
train = train[~train['item_id'].isin(list(item_to_drop.index))] # removing the items from train dataset
train.shape

(2923256, 6)

In [15]:
print(f'number of records removed : {(sales_train.shape[0]) - (train.shape[0])} ')

number of records removed : 12593 


In [16]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0
4,15.01.2013,0,25,2555,1099.0,1.0
5,10.01.2013,0,25,2564,349.0,1.0


In [17]:
print(f'Unique shop Ids in Train dataset : {len(train.shop_id.unique())}')
print(f'Unique shop Ids in Test dataset  : {len(sales_test.shop_id.unique())}')


Unique shop Ids in Train dataset : 60
Unique shop Ids in Test dataset  : 42


Removing datas of shop_id that are not in test dataset.

In [18]:
train = train.loc[train.shop_id.isin(sales_test.shop_id.unique())]

In [19]:
print(f'number of records removed : {(sales_train.shape[0]) - (train.shape[0])} ')

number of records removed : 531721 


In [20]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0
4,15.01.2013,0,25,2555,1099.0,1.0
5,10.01.2013,0,25,2564,349.0,1.0


In [21]:
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y')

In [22]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.0,1.0
1,2013-01-03,0,25,2552,899.0,1.0
2,2013-01-05,0,25,2552,899.0,-1.0
4,2013-01-15,0,25,2555,1099.0,1.0
5,2013-01-10,0,25,2564,349.0,1.0


In [23]:
def create_features(df):
    df = df.copy()
    df['dayofmonth'] = df.date.dt.day
    df['dayofweek'] = df.date.dt.day_of_week
    df['month'] = df.date.dt.month
    df['quarter'] = df.date.dt.quarter  
    df['year'] = df.date.dt.year
    return df

In [24]:
train = create_features(train)
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,dayofmonth,dayofweek,month,quarter,year
0,2013-01-02,0,59,22154,999.0,1.0,2,2,1,1,2013
1,2013-01-03,0,25,2552,899.0,1.0,3,3,1,1,2013
2,2013-01-05,0,25,2552,899.0,-1.0,5,5,1,1,2013
4,2013-01-15,0,25,2555,1099.0,1.0,15,1,1,1,2013
5,2013-01-10,0,25,2564,349.0,1.0,10,3,1,1,2013


In [25]:
train['revenue'] = train.item_price * train.item_cnt_day
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,dayofmonth,dayofweek,month,quarter,year,revenue
0,2013-01-02,0,59,22154,999.0,1.0,2,2,1,1,2013,999.0
1,2013-01-03,0,25,2552,899.0,1.0,3,3,1,1,2013,899.0
2,2013-01-05,0,25,2552,899.0,-1.0,5,5,1,1,2013,-899.0
4,2013-01-15,0,25,2555,1099.0,1.0,15,1,1,1,2013,1099.0
5,2013-01-10,0,25,2564,349.0,1.0,10,3,1,1,2013,349.0


In [26]:
train_raw = train.groupby(['date_block_num', 'shop_id', 'item_id']).agg(
    {'item_cnt_day': 'sum', 'month': pd.Series.mode}).reset_index()

train_raw.head()

,date_block_num,shop_id,item_id,item_cnt_day,month
0,0,2,27,1.0,1
1,0,2,33,1.0,1
2,0,2,317,1.0,1
3,0,2,438,1.0,1
4,0,2,471,2.0,1


In [27]:
x_train = train_raw.rename(columns={'item_cnt_day':'monthly_item_cnt'})
x_train.head()

,date_block_num,shop_id,item_id,monthly_item_cnt,month
0,0,2,27,1.0,1
1,0,2,33,1.0,1
2,0,2,317,1.0,1
3,0,2,438,1.0,1
4,0,2,471,2.0,1


In [28]:
x_train = x_train.join(items.item_category_id, on='item_id', how='left')
x_train.head()

,date_block_num,shop_id,item_id,monthly_item_cnt,month,item_category_id
0,0,2,27,1.0,1,19
1,0,2,33,1.0,1,37
2,0,2,317,1.0,1,45
3,0,2,438,1.0,1,45
4,0,2,471,2.0,1,49


In [29]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [30]:
test['month'] = 11
test['date_block_num'] = 34
test = test.drop(['ID'], axis=1)
test.head()

,shop_id,item_id,month,date_block_num
0,5,5037,11,34
1,5,5320,11,34
2,5,5233,11,34
3,5,5232,11,34
4,5,5268,11,34


In [31]:
test = test.join(items.item_category_id, on='item_id', how='left')
test.head()

,shop_id,item_id,month,date_block_num,item_category_id
0,5,5037,11,34,19
1,5,5320,11,34,55
2,5,5233,11,34,19
3,5,5232,11,34,23
4,5,5268,11,34,20


In [32]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor

In [213]:
features = ['month', 'shop_id', 'item_category_id']
feature_transform = OneHotEncoder(handle_unknown = 'ignore')
preprocessor = ColumnTransformer(transformers=[('categorical', feature_transform, features)])

In [62]:
y_train_full = x_train['monthly_item_cnt']
X_raw = x_train.drop(['monthly_item_cnt'], axis='columns')

In [43]:
X_raw.head()

,date_block_num,shop_id,item_id,month,item_category_id
0,0,2,27,1,19
1,0,2,33,1,37
2,0,2,317,1,45
3,0,2,438,1,45
4,0,2,471,1,49


In [214]:
# Preprocessing data
X_train_full = preprocessor.fit_transform(X_raw) 
X_test = preprocessor.transform(test)

X_train_full = X_train_full.toarray()
X_test = X_test.toarray()


In [73]:
X_train = X_train_full[:1200000]
X_valid = X_train_full[1200000:]
y_train = y_train_full[:1200000]
y_valid = y_train_full[1200000:]

In [74]:
features

['month', 'shop_id', 'item_category_id']

In [75]:
reg = XGBRegressor(base_score=0.5, booster='gbtree',    
                       n_estimators=1000,
                       early_stopping_rounds=50,
                       objective='reg:squarederror',
                       max_depth=3,
                       learning_rate=0.01)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        verbose=100)

[0]	validation_0-rmse:8.42935	validation_1-rmse:10.82751
[100]	validation_0-rmse:6.68299	validation_1-rmse:9.77447
[200]	validation_0-rmse:6.24365	validation_1-rmse:9.65514
[300]	validation_0-rmse:6.06831	validation_1-rmse:9.64011
[400]	validation_0-rmse:6.01885	validation_1-rmse:9.62999
[489]	validation_0-rmse:5.98022	validation_1-rmse:9.62888


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=50, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=3, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [76]:
predicted_df = sales_test.copy()
predicted_df['predicted_cnt_xgb'] = reg.predict(X_test)
predicted_df.head()

,ID,shop_id,item_id,predicted_cnt_xgb
0,0,5,5037,2.028458
1,1,5,5320,1.708823
2,2,5,5233,2.028458
3,3,5,5232,1.981844
4,4,5,5268,3.519194
